In [1]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
import pandas as pd
import numpy as np

Завантажимо датасет, побудуємо повний, тренувальний та тестовий сети

In [2]:
# Завантажуємо вбудований датасет
ds = Dataset.load_builtin("ml-100k", prompt=False)
full_set = ds.build_full_trainset()
train_set, test_set = train_test_split(ds, 0.1)

In [3]:
# Тестовий сет - масив тюплів user, item, raiting
# Для зменшення виводу при порівнянні різних моделей візьмемо перших 10
test_10 = test_set[:10]

Спробуємо створити першу модель та оцінити її за допомогою крос-валідації

In [4]:
model_SVD = SVD()
res = cross_validate(model_SVD, ds, measures=["rmse", "mae", "mse"], cv=5, verbose=True)

Evaluating RMSE, MAE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9318  0.9419  0.9406  0.9386  0.9317  0.9369  0.0043  
MAE (testset)     0.7350  0.7432  0.7385  0.7407  0.7348  0.7384  0.0033  
MSE (testset)     0.8683  0.8872  0.8847  0.8809  0.8681  0.8778  0.0081  
Fit time          0.96    0.99    1.04    1.03    1.03    1.01    0.03    
Test time         0.15    0.18    0.21    0.12    0.19    0.17    0.03    


Оцінимо також інші алгоритми за тими самими метриками та умовами

In [6]:
models_list = [SVDpp(), NMF()]

for model in models_list:
    res = cross_validate(model, ds, measures=["rmse", "mae", "mse"], cv=5, verbose=True)

Evaluating RMSE, MAE, MSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9215  0.9186  0.9244  0.9173  0.9086  0.9181  0.0053  
MAE (testset)     0.7239  0.7188  0.7249  0.7202  0.7129  0.7201  0.0043  
MSE (testset)     0.8492  0.8438  0.8544  0.8415  0.8256  0.8429  0.0098  
Fit time          18.63   19.49   18.74   19.72   19.62   19.24   0.46    
Test time         3.48    3.31    3.81    4.27    3.55    3.68    0.33    
Evaluating RMSE, MAE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9659  0.9662  0.9556  0.9708  0.9625  0.9642  0.0051  
MAE (testset)     0.7601  0.7604  0.7497  0.7607  0.7557  0.7573  0.0042  
MSE (testset)     0.9330  0.9334  0.9131  0.9425  0.9265  0.9297  0.0097  
Fit time          1.62    1.80    1.66    2.00    1.76    1.77    0.13    
Test time         0.12    0.11    0.13    0.14    0.21 

Як бачимо, "з коробки" SVD++ показує найкращі результати по точності, однак час тренування на порядок вищий ніж у інших алгоритмів.
NMF показав найгірші результати: його точність найнижча, а час тренування не кращий ніж у SVD.

Отже, зупинимось на двох алгоритмах (SVD та SVD++): в залежності від задачі нам може знадобитись висока швидкість при прийнятній точності чи висока точність незважаючи на час тренування. 

Спробуємо підібрати кращі гіперпараметри (на наше щастя для двох алгоритмів вони майже не відрізняються)

In [15]:
n_factors_list = [50, 100, 200]
n_epochs_list = [10, 20, 50]
lr_list = [0.01, 0.005, 0.001, 0.0001]
reg_list = [0.3, 0.02, 0.005]
algos = [SVD, SVDpp]

time_measure = 2 # Ми шукатимемо найточнішу модель і найточнішу серед швидких, швидкою будемо вважати модель, яка фітиться менше 2 секунд

best_rapid = {"algo": None, "fit_time": 100, "n_factors": 0, "n_epochs": 0, "lr": 0, "reg_all": 0, "RMSE": 500}
best_acc = {"algo": None, "fit_time": 100, "n_factors": 0, "n_epochs": 0, "lr": 0, "reg_all": 0, "RMSE": 500}

for algo in algos:
    for n_factors in n_factors_list:
        for n_epochs in n_epochs_list:
            for lr in lr_list:
                for reg_all in reg_list:
                    if True==False:
                        cur_algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr, reg_all=reg_all)
                    cur_algo = algo(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr, reg_all=reg_all)
                    res = cross_validate(cur_algo, ds, measures=["rmse", "mae", "mse"], cv=5, verbose=False)
                    mean_time = np.array(res["fit_time"]).mean()
                    mean_rmse = np.array(res["test_rmse"]).mean()

                    if mean_rmse < best_acc["RMSE"]:
                        best_acc["algo"] = cur_algo.__class__.__name__
                        best_acc["fit_time"] = mean_time
                        best_acc["lr"] = lr
                        best_acc["n_epochs"] = n_epochs
                        best_acc["n_factors"] = n_factors
                        best_acc["reg_all"] = reg_all
                        best_acc["RMSE"] = mean_rmse

                        print("Best accuracy")
                        print(best_acc)
                    
                    if mean_time < time_measure and mean_rmse < best_rapid["RMSE"]:
                        best_rapid["algo"] = cur_algo.__class__.__name__
                        best_rapid["fit_time"] = mean_time
                        best_rapid["lr"] = lr
                        best_rapid["n_epochs"] = n_epochs
                        best_rapid["n_factors"] = n_factors
                        best_rapid["reg_all"] = reg_all
                        best_rapid["RMSE"] = mean_rmse

                        print("Best rapid")
                        print(best_rapid)



Best accuracy
{'algo': 'SVD', 'fit_time': 0.4225654602050781, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.3, 'RMSE': 0.9526904571862331}
Best rapid
{'algo': 'SVD', 'fit_time': 0.4225654602050781, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.3, 'RMSE': 0.9526904571862331}
Best accuracy
{'algo': 'SVD', 'fit_time': 0.43748135566711427, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9323438451004076}
Best rapid
{'algo': 'SVD', 'fit_time': 0.43748135566711427, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9323438451004076}
Best accuracy
{'algo': 'SVDpp', 'fit_time': 19.810650062561034, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9197139154912118}


Збережемо вивід останнього кроку, адже ще раз чекати 18 годин - задоволення таке собі


Best accuracy

{'algo': 'SVD', 'fit_time': 0.4225654602050781, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.3, 'RMSE': 0.9526904571862331}

Best rapid

{'algo': 'SVD', 'fit_time': 0.4225654602050781, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.3, 'RMSE': 0.9526904571862331}

Best accuracy

{'algo': 'SVD', 'fit_time': 0.43748135566711427, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9323438451004076}

Best rapid

{'algo': 'SVD', 'fit_time': 0.43748135566711427, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9323438451004076}

Best accuracy

{'algo': 'SVDpp', 'fit_time': 19.810650062561034, 'n_factors': 50, 'n_epochs': 10, 'lr': 0.01, 'reg_all': 0.02, 'RMSE': 0.9197139154912118}

Також спробуємо знайти найоптимальніші параметри з використанням GridSearchCV.

Враховуючи досвід попереднього кроку (тривалість пошуку), обмежимо кількість параметрів пошуку. 

In [20]:
params_gs = {"lr_all": [0.001, 0.01], "n_epochs": [10, 50, 100], "reg_all": [0.2, 0.002, 0.0001]}

from surprise.model_selection import GridSearchCV
gs = GridSearchCV(SVD, params_gs, measures=["RMSE"], cv=5, joblib_verbose=2, n_jobs=-1)

gs.fit(ds)

print(f"Best score {gs.best_score}")
print(f"Best params {gs.best_params}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   36.2s


Best score {'rmse': 0.9307776604070455}
Best params {'rmse': {'lr_all': 0.01, 'n_epochs': 100, 'reg_all': 0.2}}


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  2.8min finished


Отже, маємо 90 виконаних завдань, що відповідає 2 (кількість варіантів lr) * 3 (кількість варіантів епох) * 3 (кількість варіантів регуляризації) * 5 (кількість фолдів). З цього ми можемо:
1. Опосередковано визначити середній час навчання (близько 1,5 секунди)
2. Приблизно (з точністю до слона, адже на різних гіперпараметрах час навчання може суттєво різнитись) оцінити час пошуку параметрів, знаючи середній час навчання для одного набору параметрів

Отже, звузимо пошук по сітці для SVD++ з тим, щоб він виконався у прийнятні терміни

In [21]:
params_gs = {"lr_all": [0.001, 0.01], "n_epochs": [10, 50], "reg_all": [0.2, 0.002]}

from surprise.model_selection import GridSearchCV
gs = GridSearchCV(SVDpp, params_gs, measures=["RMSE"], cv=3, joblib_verbose=10, n_jobs=-1)

gs.fit(ds)

print(f"Best score {gs.best_score}")
print(f"Best params {gs.best_params}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  12 out of  24 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed:  2.2min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:  2.2min remaining:   43.5s
[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:  3.4min remaining:   28.8s


Best score {'rmse': 0.925687827976405}
Best params {'rmse': {'lr_all': 0.01, 'n_epochs': 10, 'reg_all': 0.002}}


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.4min finished


Результат невтішний: пошук виконався швидко, але найкращий результат гірший, ніж у пошуку, що виконувався вручну. Я пов'язав би це з кількістю фолдів: на кожному етапі навчальна вибірка менша, а отже і результат навчання гірший. Спробуємо ще раз

In [22]:
params_gs = {"lr_all": [0.001, 0.01], "n_epochs": [10, 50], "reg_all": [0.2, 0.002]}

from surprise.model_selection import GridSearchCV
gs = GridSearchCV(SVDpp, params_gs, measures=["RMSE"], cv=5, joblib_verbose=10, n_jobs=-1)

gs.fit(ds)

print(f"Best score {gs.best_score}")
print(f"Best params {gs.best_params}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:  5.9min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:  7.4min remaining:  1.1min


Best score {'rmse': 0.9160688551927472}
Best params {'rmse': {'lr_all': 0.01, 'n_epochs': 10, 'reg_all': 0.002}}


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  8.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  8.8min finished


In [29]:
res_df = pd.DataFrame.from_dict(gs.cv_results)
res_df


,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_lr_all,param_n_epochs,param_reg_all
0,0.990761,0.983251,0.978748,0.982933,0.979832,0.983105,0.004204,7,22.328686,0.317584,9.500512,0.108123,"{'lr_all': 0.001, 'n_epochs': 10, 'reg_all': 0.2}",0.001,10,0.200
1,0.973245,0.965921,0.962332,0.965249,0.963135,0.965976,0.003866,6,23.758709,1.452256,9.233275,0.466685,"{'lr_all': 0.001, 'n_epochs': 10, 'reg_all': 0...",0.001,10,0.002
2,0.957953,0.949109,0.947521,0.950527,0.947475,0.950517,0.003886,5,136.842884,1.156854,12.482302,0.818488,"{'lr_all': 0.001, 'n_epochs': 50, 'reg_all': 0.2}",0.001,50,0.200
3,0.927350,0.919366,0.921359,0.920666,0.918815,0.921511,0.003056,2,148.332527,8.162144,11.620702,0.603335,"{'lr_all': 0.001, 'n_epochs': 50, 'reg_all': 0...",0.001,50,0.002
4,0.953919,0.944718,0.944164,0.946944,0.944627,0.946874,0.003652,4,30.592524,1.038900,12.974686,0.909886,"{'lr_all': 0.01, 'n_epochs': 10, 'reg_all': 0.2}",0.010,10,0.200
5,0.921112,0.915428,0.914689,0.914902,0.914213,0.916069,0.002552,1,32.294980,0.397992,12.344384,0.626064,"{'lr_all': 0.01, 'n_epochs': 10, 'reg_all': 0....",0.010,10,0.002
6,0.938879,0.929322,0.930130,0.932355,0.930168,0.932171,0.003502,3,160.518949,0.942412,11.635493,0.182898,"{'lr_all': 0.01, 'n_epochs': 50, 'reg_all': 0.2}",0.010,50,0.200
7,1.051366,1.042059,1.046318,1.045654,1.054994,1.048078,0.004558,8,116.525917,32.270637,7.012938,2.381240,"{'lr_all': 0.01, 'n_epochs': 50, 'reg_all': 0....",0.010,50,0.002


cv_results надає нам додаткову інформацію по результатам оцінки, зокрема час навчання (якщо цей параметр буде критичним) та стандартне відхилення для похибки на тестових даних на фолдах (що певною мірою говорить про якість навчання)

Наразі можемо обрати 2 моделі для подальших тестів:
1. SVD n_factors: 50, n_epochs: 10, lr: 0.01, reg_all: 0.02, RMSE: 0.9323438451004076 як найшвидшу, якщо величина похибки буде прийнятною
2. SVDpp lr_all: 0.01, n_epochs: 10, reg_all: 0.002, rmse: 0.9160688551927472 як найточнішу

In [30]:
# Отже сворюємо моделі та вчимо їх
rapid_SVD = SVD(n_factors=50, n_epochs=10, lr_all=0.01, reg_all=0.02)
rapid_SVD.fit(train_set)

In [31]:
acc_SVDpp = SVDpp(n_epochs=10, lr_all=0.01, reg_all=0.02)
acc_SVDpp.fit(train_set)

In [33]:
# Здійснимо передбачення і оцінку rj;yj] моделі на тестових даних
print("Швидкий SVD")
pred_train_svd = rapid_SVD.test(test_set)

accuracy.rmse(pred_train_svd)
accuracy.mse(pred_train_svd)

print("Точний SVD++")
pred_train_svdpp = acc_SVDpp.test(test_set)

accuracy.rmse(pred_train_svdpp)
accuracy.mse(pred_train_svdpp)

Швидкий SVD
RMSE: 0.9372
MSE: 0.8784
Точний SVD++
RMSE: 0.9220
MSE: 0.8500


0.8500372424511963

Маючи натреновані моделі, спробуємо зробити передбачення

In [40]:
from collections import defaultdict

def get_top_predictions(predictions, n=5):
    # Створюємо словник, елементами якого є список 
    top_pred = defaultdict(list)
    
    # Передбачення - список тюплів, з яких нам потрібні uid, iid, est
    for uid, iid, true_rating, est, smth in predictions:
        top_pred[uid].append((iid, est))

    # На даному етапі маємо словник, ключами якого є айді користувачів, а даними список тюплів айді фільму та передбаченої оцінки
    # отже нам слід відсортувати кожен зі списків по оцінці і взяти n найвищих
    for uid, pred_list in top_pred.items():
        pred_list.sort(key=lambda x:x[1], reverse=True)
        top_pred[uid] = pred_list[:n]

    return top_pred




In [51]:
small_test_set = [i for i in full_set.build_anti_testset() if i[0]=="196"]

pred_rapid = rapid_SVD.test(small_test_set)
pred_rapid_top = get_top_predictions(pred_rapid, 10)
pred_rapid_top

defaultdict(list,
            {'196': [('12', 4.64099691300788),
              ('483', 4.568624300933987),
              ('408', 4.530250855743716),
              ('178', 4.50927436337415),
              ('64', 4.496384802570373),
              ('318', 4.438268896682861),
              ('50', 4.438240021587974),
              ('427', 4.433056854893131),
              ('169', 4.418319642759491),
              ('114', 4.417667707949969)]})

In [52]:
pred_acc_SVDpp = acc_SVDpp.test(small_test_set)
pred_acc_top = get_top_predictions(pred_acc_SVDpp, 10)
pred_acc_top

defaultdict(list,
            {'196': [('178', 4.474204903092372),
              ('603', 4.468404627648023),
              ('64', 4.462157113611034),
              ('427', 4.445518155308937),
              ('483', 4.443096290683881),
              ('357', 4.424675802069754),
              ('318', 4.412330169543648),
              ('169', 4.40261665262087),
              ('513', 4.400071662045988),
              ('408', 4.3837670860165066)]})

Деякі передбачення для різних моделей збігаються

Спробуємо задачу з зірочкою